<a href="https://colab.research.google.com/github/MJMortensonWarwick/Programming_and_Big_Data_Analytics_2425/blob/main/3_03_key_value_stores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.03 Key-value Stores with TinyDB
This tutorial gives a basic introduction to working with key-value (KV) stores (or document DBs). We will be working with [TinyDB](https://tinydb.readthedocs.io/en/latest/index.html), an in-memory Python database, which is particularly attractive here as it is, as the name suggests, pretty small and lightweight.

We will begin with the relevant installs:

In [ ]:
!pip install tinydb
!pip install faker
!pip install python-lorem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


As you may infer from the pacakages installed, we will run something similar to one of our DuckDB examples - specifically building a database using fake data generated by Faker:

In [ ]:
import random
from faker import Faker
import pandas as pd
from lorem import paragraph
import itertools

fake = Faker()

def get_person():
  person = {}
  person['id'] = random.randrange(1000,9999999999999)
  person['first_name'] = fake.first_name()
  person['last_name'] = fake.last_name()
  person['email'] = fake.unique.ascii_email()
  person['company'] = fake.company()
  person['phone'] = fake.phone_number()
  person['review'] = list(itertools.islice(paragraph(count=1), 1))
  return person

personlist = []
for x in range(100):
  personlist.append(get_person())

df = pd.DataFrame.from_dict(personlist)
df.head()

,id,first_name,last_name,email,company,phone,review
0,8548560491567,Emily,Sparks,cheryl85@gmail.com,Farrell Ltd,001-178-203-5651,"[Veniam laborum nulla sed amet aliqua, est sin..."
1,4426360358696,Shannon,Walker,teresamartinez@soto-simmons.info,Walker Ltd,001-145-073-8604x504,"[Officia reprehenderit est elit laboris, ipsum..."
2,6527301016016,Alexander,Ingram,hnichols@edwards.com,Hughes Group,(250)093-0524,[Exercitation sunt dolore deserunt cupidatat c...
3,6619028137069,Brian,Duncan,alexis70@yahoo.com,Phillips-Brown,2776805400,[Esse consectetur amet aute nulla ipsum sit es...
4,8428438754176,Gina,Peters,dillondeleon@yahoo.com,"Sanchez, Little and Munoz",1475179965,"[Aute qui reprehenderit labore, et magna in ex..."


Everything here is the same except we have also add a text column (using lorem ipsum). As before we have created this as a Pandas dataframe, but like most KV stores, TinyDB prefers data stored as a dictionary:

In [ ]:
fake_data = df.to_dict(orient='records')
fake_data

[{'id': 8548560491567,
  'first_name': 'Emily',
  'last_name': 'Sparks',
  'email': 'cheryl85@gmail.com',
  'company': 'Farrell Ltd',
  'phone': '001-178-203-5651',
  'review': ['Veniam laborum nulla sed amet aliqua, est sint sint consectetur eiusmod ex do lorem. Amet occaecat amet sit. Enim pariatur nostrud magna commodo. In enim eiusmod ipsum non qui nisi. Consectetur officia non anim.']},
 {'id': 4426360358696,
  'first_name': 'Shannon',
  'last_name': 'Walker',
  'email': 'teresamartinez@soto-simmons.info',
  'company': 'Walker Ltd',
  'phone': '001-145-073-8604x504',
  'review': ['Officia reprehenderit est elit laboris, ipsum consequat esse ad tempor consectetur labore enim. Excepteur sed minim magna commodo irure proident reprehenderit, tempor nulla in do consectetur. Et commodo lorem culpa. Cupidatat ex enim ullamco do do. Nisi enim eu magna voluptate. Quis veniam tempor consequat anim.']},
 {'id': 6527301016016,
  'first_name': 'Alexander',
  'last_name': 'Ingram',
  'email': '

With this transform in place we can load the data into our database. You may note the database itself is specified as JSON format:

In [ ]:
from tinydb import TinyDB, Query

db = TinyDB('db.json')

for record in fake_data:
  db.insert(record)

We can check this has worked with a simple Python loop:

In [ ]:
for item in db:
  print(item)

{'id': {'0': 8548560491567, '1': 4426360358696, '2': 6527301016016, '3': 6619028137069, '4': 8428438754176, '5': 818960350733, '6': 2334732784028, '7': 3657881027455, '8': 6749639524081, '9': 6375023396131, '10': 3097277562145, '11': 3113108040389, '12': 3774925961362, '13': 4121727873765, '14': 8340947142649, '15': 584303647819, '16': 8407646734895, '17': 6248363451299, '18': 3047032711727, '19': 9969517588316, '20': 8334378593169, '21': 3112626853607, '22': 8701577584259, '23': 6727503492027, '24': 1777019421736, '25': 6614976431713, '26': 530632407496, '27': 3051911100123, '28': 651904123888, '29': 4354379732886, '30': 73880747032, '31': 7782931368534, '32': 6771678536112, '33': 6126752688440, '34': 2278108053635, '35': 4538040946757, '36': 3057272431325, '37': 4320958967860, '38': 4971197497666, '39': 37808870899, '40': 9436337377705, '41': 4422625325930, '42': 4422681212558, '43': 6527572160164, '44': 8415727160437, '45': 9540912474595, '46': 4967921613639, '47': 2552778432433, '4

With our database setup, we can start to query our records. In TinyDB we do this by creating a query object:

In [ ]:
User = Query() # query object

db.search(User.first_name == 'Chad') # adapt based on your data

[{'id': 8821397154913,
  'first_name': 'Chad',
  'last_name': 'Ford',
  'email': 'lovetiffany@hotmail.com',
  'company': 'Flores Ltd',
  'phone': '(067)787-8022',
  'review': ['Consequat excepteur culpa eiusmod anim minim est lorem. Veniam amet sunt eiusmod cupidatat proident magna. Est aliqua aliquip ipsum duis anim aute. Veniam reprehenderit cillum labore pariatur mollit incididunt, aliqua exercitation voluptate officia, adipiscing ea officia mollit consectetur. Sed culpa fugiat dolore dolore cillum anim veniam. Veniam ipsum cillum sit officia.']}]

We can also add new data in dictionary/JSON-like format:

In [ ]:
db.insert({'id': 123, 'first_name': 'Amir', 'star_sign': 'Dog', 'review': 'I do not speak Latin.'})

102

And retrieve the data as before:

In [ ]:
db.search(User.id == 123)

[{'id': 123,
  'first_name': 'Amir',
  'star_sign': 'Dog',
  'review': 'I do not speak Latin.'}]

One thing to note here is that our new record does not follow the schema we may infer from the original dataset (i.e. the original data all used the same columns/fields). Here many of those fields are missing and we have the new field 'star_sign'.

This demonstrates the extra flexibility we get with a KV store over a relational model. We can also query our database to get all records that have a specific field:

In [ ]:
db.search(User.star_sign.exists())

[{'id': 123,
  'first_name': 'Amir',
  'star_sign': 'Dog',
  'review': 'I do not speak Latin.'}]

This gives a basic intro into KV (and document) stores. While there are many competing brands/solutions, the common themes are the dictionary-like structure (key-value pairs) and flexibility to accept any fields (keys).